# AI Powered HR Assistant for Nestle Policy 

### 1. Import Important Packages 

In [ ]:
import os
import openai
from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI

from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

### 2. Setup OpenAI API calling Environment

In [ ]:
# Setup open api Key
# Load environment variables from .env file if present
load_dotenv()

# Get the OpenAI API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

print("OpenAI API key loaded successfully.")

OpenAI API key loaded successfully.


### 3. Load and process Nestle Policy data 

In [ ]:
pdf_path = './Dataset/the_nestle_hr_policy_pdf_2012.pdf'
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Loaded {len(documents)} documents from the PDF.")
print(f"First page preview: {documents[0].page_content[:2000]}...")

Loaded 8 documents from the PDF.
First page preview: Policy
Mandatory
September  2012
The Nestlé  
Human Resources Policy...


### 4. Vector Store initialization and Embeddings generation --

In [ ]:
# Intialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

In [ ]:
# Create or load the Chroma vector store using Chroma
vector_store = Chroma.from_documents(
    documents, 
    embeddings, 
    persist_directory="./chroma_db"
    )

In [ ]:
print("Vector store created and embeddings generated.")
print(f"Number of documents in vector store: {vector_store._collection.count()}")

Vector store created and embeddings generated.
Number of documents in vector store: 48


### 5. Intialize LLM from OpenAI

In [ ]:
# Initialize ChatOpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=500)

In [ ]:
print("llm ChatOpenAI model loaded")

llm ChatOpenAI model loaded


In [ ]:
# Create a retrieval-based QA chain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
print("Retrieval-based QA chain created successfully.")
print(f"Retriever configured to return top {retriever.search_kwargs['k']} similar documents.")
print(f"Retriever search type: {retriever.search_type}")
print(f"Retriever details: {retriever}")
print(f"Vector store details: {vector_store}")
print(f"Retriever : {retriever.get_relevant_documents}")

Retrieval-based QA chain created successfully.
Retriever configured to return top 3 similar documents.
Retriever search type: similarity
Retriever details: tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x74f8eafba600> search_kwargs={'k': 3}
Vector store details: <langchain_chroma.vectorstores.Chroma object at 0x74f8eafba600>
Retriever : <bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x74f8eafba600>, search_kwargs={'k': 3})>


### 6. Create PROMPT Template for Custom Prompt for Chat Assistant Model for NESTLE ~

In [ ]:
template = """

Your are Helpful AI assistant for NESTLE HR policies. Your role is to help Employee to provide them accurate information in human understandable format. 
If you can not find the answer from the provided context, simply state that "I don't have that information". Do not make up an answer.

Use the following context to answer the question - 
Context = {context}

Question = {question}

HR Assistant Reponse:
"""



In [ ]:
PROMPT = PromptTemplate(
    template=template, 
    input_variables=["context", "question"]
)

In [ ]:
# Create Retrieval QA Chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

### 7. Verify & Test HR Assistant Response ~

In [15]:
# Test function for the HR assistant
def test_hr_assistant(question):
    """Test function to query the HR assistant"""
    try:
        result = qa_chain({"query": question})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Test with sample questions
test_questions = [
    "What is Nestlé's policy on employee training and development?",
    "How does Nestlé handle performance management?",
    "What are the total rewards offered by Nestlé?"
]

print("Testing HR Assistant with sample questions:\n")
for i, question in enumerate(test_questions, 1):
    print(f"Q{i}: {question}")
    answer = test_hr_assistant(question)
    print(f"A{i}: {answer}\n")
    print("-" * 80)

Testing HR Assistant with sample questions:

Q1: What is Nestlé's policy on employee training and development?
A1: Nestlé's policy on employee training and development emphasizes a culture of continuous learning and growth. The company determines training priorities and encourages employees at all levels to upgrade their knowledge and skills. Training and development actions are shared responsibilities between employees, line managers, and Human Resources. Experience and on-the-job training are primary sources of learning, with managers guiding and coaching employees to succeed in their current positions. Additionally, Nestlé offers a comprehensive range of training activities and methodologies to support everyone's learning and growth. Corporate leadership programs are also available to develop and retain qualified management. Practices such as lateral professional development, extension of responsibilities, and cross-functional teams are encouraged to acquire additional skills and en

### 8. Create Gradio Interface for QA with HR Assistance in Nestle Company 

In [19]:
import gradio as gr

In [20]:
def hr_chatbot(message, history):
    """
    Main chatbot function for Gradio interface
    """
    try:
        # Get response from the QA chain
        result = qa_chain({"query": message})
        response = result["result"]
        
        # Add some context about the source
        if result.get("source_documents"):
            response += "\n\n📋 *Based on Nestlé HR Policy Document (2012)*"
        
        return response
    
    except Exception as e:
        return f"I apologize, but I encountered an error while processing your question: {str(e)}. Please try rephrasing your question or contact HR directly for assistance."

# Create the Gradio interface
def create_gradio_interface():
    """
    Create and configure the Gradio chatbot interface
    """
    
    # Custom CSS for better styling
    custom_css = """
    .gradio-container {
        font-family: 'Arial', sans-serif;
    }
    .chat-message {
        padding: 10px;
        margin: 5px;
        border-radius: 10px;
    }
    """
    
    # Create the chatbot interface
    with gr.Blocks(css=custom_css, title="Nestlé HR Assistant") as demo:
        
        # Header
        gr.Markdown(
            """
            # 🏢 Nestlé AI-Powered HR Assistant
            
            Welcome to your intelligent HR assistant! I can help you find information about:
            - 📚 HR policies and procedures
            - 💼 Employment conditions and benefits
            - 🎯 Performance management and development
            - 🌟 Training and learning opportunities
            - 🤝 Employee relations and workplace culture
            
            Ask me anything about Nestlé's HR policies!
            """
        )
        
        # Chatbot interface
        chatbot = gr.ChatInterface(
            hr_chatbot,
            title="HR Policy Assistant",
            description="Ask questions about Nestlé's HR policies and get instant answers!",
            examples=[
                "What is Nestlé's approach to employee development?",
                "How does Nestlé handle performance evaluations?",
                "What are the total rewards offered to employees?",
                "What is Nestlé's policy on work-life balance?",
                "How does Nestlé support diversity and inclusion?",
                "What training opportunities are available?"
            ],
            # retry_btn="🔄 Retry",
            # undo_btn="↩️ Undo",
            # clear_btn="🗑️ Clear Chat"
        )
        
        # Footer
        gr.Markdown(
            """
            ---
            **Note:** This assistant is based on Nestlé's HR Policy document from 2012. 
            For the most current policies or specific situations, please contact your HR department directly.
            
            *Powered by OpenAI GPT-3.5 Turbo and LangChain*
            """
        )
    
    return demo

print("Gradio interface configuration complete!")

Gradio interface configuration complete!


In [21]:
# Create and launch the Gradio interface
if __name__ == "__main__":
    print("🚀 Launching Nestlé HR Assistant...")
    
    # Create the interface
    demo = create_gradio_interface()
    
    # Launch the application
    demo.launch(
        share=True,  # Creates a public link
        server_name="0.0.0.0",  # Makes it accessible from other devices
        server_port=7860,  # Default Gradio port
        show_error=True,  # Show detailed error messages
        debug=True  # Enable debug mode
    )
    
    print("\n✅ HR Assistant is now running!")
    print("📱 Access the interface through the provided URL")
    print("🔗 Share the public link with your team members")

🚀 Launching Nestlé HR Assistant...


/home/neogen/PROJECTS/python-env/SRC/IITK-AIML/end-proj/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://58a5d7aa726b8d51a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://58a5d7aa726b8d51a6.gradio.live

✅ HR Assistant is now running!
📱 Access the interface through the provided URL
🔗 Share the public link with your team members
